In [4]:
import json
import csv
import math
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon

In [15]:

def deg2num(lat_deg, lon_deg, zoom):
    lat_rad = math.radians(lat_deg)
    n = 2.0 ** zoom
    xtile = (lon_deg + 180.0) / 360.0 * n
    ytile = (1.0 - math.log(math.tan(lat_rad) + (1 / math.cos(lat_rad))) / math.pi * n / 2.0)
    return (xtile, ytile)

# Lue maakuntien rajat
with open('maakuntarajat.json', 'r', encoding='utf-8') as f:
    maakunnat = json.load(f)

# Lue asukastiheydet
tiheydet = {}
max_tiheys = 0
with open('maakuntatiheydet.csv', 'r', encoding='utf-8') as f:
    reader = csv.reader(f)
    next(reader)  # Ohita otsikkorivi
    for row in reader:
        nimi = row[0]
        tiheys = float(row[1])
        tiheydet[nimi] = tiheys
        if tiheys > max_tiheys:
            max_tiheys = tiheys

# Alusta kuva
plt.figure()
ax = plt.gca()
ax.set_aspect('equal', adjustable='box')
ax.set_axis_off()

# Käy läpi maakunnat ja piirrä polygonit
min_x, max_x = float('inf'), -float('inf')
min_y, max_y = float('inf'), -float('inf')

for nimi, polygonit in maakunnat.items():
    tiheys = tiheydet.get(nimi, 0)
    harmaasavy = (1 - tiheys / max_tiheys) / 1.05
    harmaasavy_str = str(harmaasavy)
    
    for poly in polygonit:
        # Muunna koordinaatit
        coords = [deg2num(lat, lon, 14) for lon, lat in poly]
        x_coords = [x for x, y in coords]
        y_coords = [y for x, y in coords]
        
        # Päivitä ääriarvot
        min_x = min(min_x, min(x_coords))
        max_x = max(max_x, max(x_coords))
        min_y = min(min_y, min(y_coords))
        max_y = max(max_y, max(y_coords))
        
        # Piirrä polygoni
        patch = Polygon(coords, edgecolor="white", facecolor=harmaasavy_str)
        ax.add_patch(patch)

# Aseta rajat
ax.set_xlim(math.floor(min_x), math.ceil(max_x))
ax.set_ylim(math.ceil(max_y), math.floor(min_y))

# Tallenna kuva
plt.savefig("suomi.png", bbox_inches='tight')
plt.close()